In [9]:
import pandas as pd
from sklearn.cluster import DBSCAN
import folium

In [10]:
# Load the CSV file into a DataFrame
df = pd.read_csv("data/restaurants_data_analysis.csv")

# Filter Singapore only for example
df = df[df["country"] == "Singapore"]


/var/folders/48/plb95v9d0gxd6mh1q0n645v40000gn/T/ipykernel_97943/1733722491.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/restaurants_data_analysis.csv")


In [11]:
from folium.plugins import MarkerCluster

# make and save map
def save_map(record, file_save):
	map_center = [record['latitude'].mean(), record['longitude'].mean()]
	mymap = folium.Map(location=map_center, zoom_start=12)

	marker_cluster = MarkerCluster().add_to(mymap)
	for _, row in record.iterrows():
		folium.Marker(
			location=[row['latitude'], row['longitude']],
		).add_to(marker_cluster)

	mymap.save(file_save)

In [12]:
# save restaurants map to html before grouped
save_map(df, 'maps/restaurants_map.html')

In [13]:
coords = df[['latitude', 'longitude']].values

# Value 6371.0088 for DBSCAN
kms_per_radian = 6371.0088
epsilon = 1 / kms_per_radian  # 1 km for example

# Apply DBSCAN on locations
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(coords)

# Add .NO for clusters
df['cluster'] = db.labels_

# cluster is -1 if restaurent not include to group
grouped_df = df[df['cluster'] != -1]


# Save all date in clusters variable
clusters = []
for cluster_number in set(grouped_df['cluster']):
    clusters.append(grouped_df[grouped_df['cluster'] == cluster_number])

In [14]:
# Sorted by restaurents len in one region
sorted_groups = sorted(clusters, key=lambda items: len(items), reverse=1)

In [15]:

_record = list(map(lambda group: (group["latitude"].mean(), group["longitude"].mean()), sorted_groups))

record = pd.DataFrame(_record, columns=['latitude', 'longitude'])

In [16]:
# Save clusters location to html map
save_map(record, 'maps/restaurants_cluster_map.html')

![example 0](screenshot/0_example.png)

In [18]:
sorted_groups[0].head() # Nice case for example (include 59 restaurants in one region)
# found 'restaurents' cluster on 'Populations' cluster

,budget,is_new_until,latitude,longitude,minimum_delivery_time,minimum_order_amount,minimum_pickup_time,name,post_code,rating,...,delivery_provider,is_active,is_new,is_promoted,city,timezone,dine_in,main_cuisine,country,cluster
76029,1,2020-06-07T00:00:00Z,1.428627,103.835870,4930,0,10,Suparakki Ramen (Yishun),768019,4.6,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Japanese,Singapore,950
76054,2,2022-11-24T00:00:00Z,1.429848,103.835554,4930,0,15,WingStreet by Pizza Hut (Northpoint),769098,0.0,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Pizza,Singapore,950
76138,2,2022-03-13T00:00:00Z,1.428309,103.836513,4935,0,15,Skillet By The Fat Boss (Northpoint),768019,4.9,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Western,Singapore,950
76146,2,2022-04-02T00:00:00Z,1.429703,103.835724,4930,0,10,Renee's Western Cuisine @ Kopitiam Northpoint,769098,4.1,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Western,Singapore,950
76148,2,2020-07-17T00:00:00Z,1.428860,103.835901,4930,0,10,Western Cuisine (Bagus - Northpoint City),768019,4.6,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Western,Singapore,950


![example 15](screenshot/15_example.png)

In [20]:
sorted_groups[15].head() # region (include 23 restaurants for example)
# found 'restaurents' cluster on 'Populations' cluster

,budget,is_new_until,latitude,longitude,minimum_delivery_time,minimum_order_amount,minimum_pickup_time,name,post_code,rating,...,delivery_provider,is_active,is_new,is_promoted,city,timezone,dine_in,main_cuisine,country,cluster
71178,2,2018-05-18T00:00:00Z,1.351040,103.872514,4930,0,10,Guan Chee HK Roast (NEX),556083,4.3,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Chinese,Singapore,138
71353,3,2020-10-21T00:00:00Z,1.350585,103.872665,4930,0,10,ROYCE' (NEX),556083,4.9,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,NaN,Singapore,138
71430,3,2018-05-12T00:00:00Z,1.350950,103.872530,4930,0,5,Chai Fish Soup (NEX),556083,4.4,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Singaporean,Singapore,138
71623,1,2020-05-22T00:00:00Z,1.350543,103.872749,4935,0,10,Chinatown Tan's Tutu,556083,4.5,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Dessert,Singapore,138
71906,1,2022-02-10T00:00:00Z,1.350939,103.871761,4930,0,10,Shabu-GO (NEX),556083,0.0,...,platform_delivery,True,False,False,Singapore,Asia/Singapore,False,Japanese,Singapore,138
